In [0]:
import scandir
import os
import sys
sys.path.append("../include_utils")
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
import cyvcf

In [0]:
from hdfstorehelper import HDFStoreHelper

In [0]:
notimputed_dir = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/notimputed/"
imputed_dir = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/beagle40/"

In [0]:
hdf_notimputed = HDFStoreHelper(os.path.join(notimputed_dir, "gypsy_samtools12_snps.vcf.gz.hd5"))
hdf_imputed = hdf2 = HDFStoreHelper(os.path.join(imputed_dir, "gypsy_samtools12_imputed40.vcf.gz.hd5"))

In [0]:
good_snps_notimputed = hdf_notimputed.get('good_snps')

In [0]:
good_snps_imputed = hdf_imputed.get('good_snps')

In [0]:
good_snps_imputed.shape, good_snps_notimputed.shape

In [0]:
isect_index = set.intersection(set(good_snps_imputed.index), set(good_snps_notimputed.index))

In [0]:
isect_index = sorted(isect_index)

In [0]:
len(isect_index)

In [0]:
good_snps_isect_notimputed = good_snps_notimputed.ix[isect_index]
good_snps_isect_imputed = good_snps_imputed.ix[isect_index]

In [0]:
hdf_notimputed.put('good_snps_isect', good_snps_isect_notimputed)

In [0]:
hdf_imputed.put('good_snps_isect', good_snps_isect_imputed)

In [0]:
good_snps_isect_notimputed.shape, good_snps_isect_imputed.shape

In [0]:
good_snps_isect_notimputed.head()

In [0]:
good_snps_isect_imputed.head()

In [0]:
isect_notimputed_vcf = os.path.join(notimputed_dir, "isect.vcf")
isect_imputed_vcf = os.path.join(imputed_dir, "isect.vcf")
good_not_vcf = os.path.join(notimputed_dir, "snps.vcf.gz")
good_imputed_vcf = os.path.join(imputed_dir, "imputed40.vcf.gz")

In [0]:
files = {"not": {"good": good_not_vcf, 
                 "isect": isect_notimputed_vcf, 
                 "filter": hdf_notimputed.get('good_snps_isect')},
        "imputed": {"good": good_imputed_vcf, 
                    "isect": isect_imputed_vcf, 
                    "filter": hdf_imputed.get('good_snps_isect')}}

In [0]:
for f in files:
    found  = 0
    reader = cyvcf.VCFReader(filename=files[f]['good'])
    out = open(files[f]['isect'], "w")
    writer = cyvcf.VCFWriter(out, reader)
    for row, data in files[f]['filter'].iterrows():
        recs = list(reader.fetch(data.contig, data.pos, data.pos))
        assert isinstance(recs[0], cyvcf.parser.Record)
        writer.write_record(recs[0])
        found += 1
    out.close()
    print f, found